In [1]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
def Cpk(mylist, usl, lsl):
    arr = np.array(mylist)
    arr = arr.ravel()
    sigma = np.std(arr)
    m = np.mean(arr)

    Cpu = float(usl - m) / (3*sigma)
    Cpl = float(m - lsl) / (3*sigma)
    Cpk = np.min([Cpu, Cpl])
    return Cpk

In [3]:
# Import DE SISO R2R history data
df=pd.read_csv('55LIMetal45Days.csv')

tool_list = np.unique(np.array(df['Tool_Id']))

# Modify target, slope, control limits
T=269
m=-11.44
FF_m=0

ucl = 289
lcl = 249

lamda_list=(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)

In [4]:
#####
target_column = 'PostMetro_target'  #Output1_target
RPA1_column = 'RPAValue_default'  #Input1_default
Time_column = 'UsedTimeStamp'   #TIME_STAMP
output_column = 'PostMetro_post'  #Output1_post
####

df[Time_column] = pd.to_datetime(df[Time_column])

df['recRPA_FF_1'] = 'na'
df['cnew_1'] = 'na'
df['new_outputs_1'] = 'na'
df['creal_1'] = 'na'
df['recRPA_1'] = 'na'
df['lamda'] = 'na'

df['recRPA_FF_2'] = 'na'
df['cnew_2'] = 'na'
df['new_outputs_2'] = 'na'
df['creal_2'] = 'na'
df['recRPA_2'] = 'na'

df['recRPA_FF_3'] = 'na'
df['cnew_3'] = 'na'
df['new_outputs_3'] = 'na'
df['creal_3'] = 'na'
df['recRPA_3'] = 'na'
#df['group_context'] = df['Tool_Id'].astype(str)+'_'+df['ModelGroup'].astype(str)
toolreal_ccpk=np.array([])
model1_ccpk = np.array([])
tool1_ccpk=np.array([])
model2_ccpk = np.array([])
tool2_ccpk=np.array([])
model3_ccpk = np.array([])
tool3_ccpk=np.array([])
real_ccpk = np.array([])
chuck_ccpk = np.array([])

lamda_simu = np.array([])

####### create empty dataframe but remain column name #########################################
df_tool = df.iloc[0:0].copy()

df = df.sort_values(by=Time_column, ascending=True)
#df_t=df.loc[df['Tool_Id']==tool_list[0]]
#df_t=df_t.reset_index(drop=True)
for p in range(len(lamda_list)):
    
    for i in range(len(tool_list)):
        lamda = lamda_list[p]
        c0=0
        df_t=df.loc[(df['Tool_Id']==tool_list[i])]
        df_t=df_t.reset_index(drop=True)
        df_t=df_t.sort_values(by=Time_column, ascending=True)
        
        if len(df_t.index)==0:
            continue
        ###################################################################################
#            toolreal_ccpk=np.append(toolreal_ccpk,tool_list[i])
#            chuck_ccpk=np.append(chuck_ccpk,chuck_list[k])
#            mean_list = np.array(df_t[output_column])
#            ccpk=Cpk(mean_list,838.2,825.8)
#            real_ccpk = np.append(real_ccpk,ccpk)
        
        ###################################################################################
        c0 = df_t[target_column][0] - m*df_t[RPA1_column][0] - FF_m*df_t['FFDisturbance1'][0]
        new_outputs = df_t[output_column][0]
        creal = new_outputs-m*df_t[RPA1_column][0] - FF_m*df_t['FFDisturbance1'][0]
        cnew = c0 + lamda*(creal - c0)
        recRPA_FF = (df_t[target_column][0] - cnew - FF_m*df_t['FFDisturbance1'][0])/m
        recRPA = (df_t[target_column][0] - cnew)/m
        df_t['recRPA_FF_1'][0] = recRPA_FF
        df_t['cnew_1'][0] = cnew
        df_t['new_outputs_1'][0] = new_outputs
        df_t['creal_1'][0] = creal
        df_t['recRPA_1'][0] = recRPA
        df_t['lamda'][0] = lamda
        
        for j in range(len(df_t.index)):
            if j == 0:
                continue
            recRPA_FF = (df_t[target_column][j] - cnew - FF_m*df_t['FFDisturbance1'][j])/m
            recRPA = (df_t[target_column][j] - cnew)/m
            new_outputs = df_t[output_column][j] + m*(recRPA_FF-df_t[RPA1_column][j])
            creal = new_outputs - m*recRPA_FF - FF_m*df_t['FFDisturbance1'][j]
            cnew = cnew + lamda*(creal - cnew)
    #        if j < len(df_t.index)-1 :
    #            recRPA_FF = (df_t[target_column][j] - cnew - FF_m*df_t['FFDisturbance1'][j+1])/m
    #            recRPA = (df_t[target_column][j] - cnew)/m
            df_t['recRPA_FF_1'][j] = recRPA_FF
            df_t['cnew_1'][j] = cnew
            df_t['new_outputs_1'][j] = new_outputs
            df_t['creal_1'][j] = creal
            df_t['recRPA_1'][j] = recRPA
            df_t['lamda'][j] = lamda
            
        df_tool = df_tool.append(df_t)
        
        ##################
#        chuck_ccpk=np.append(chuck_ccpk,chuck_list[k])
        tool1_ccpk=np.append(tool1_ccpk,tool_list[i])
        mean_list=np.array(df_t['new_outputs_1'])
        ccpk=Cpk(mean_list,ucl,lcl)
        model1_ccpk=np.append(model1_ccpk,ccpk)
        lamda_simu=np.append(lamda_simu,lamda)
        ##################
    ######CCpk for each lamda##########################
    df_lamda = df_tool.loc[df_tool.lamda==lamda_list[p]]
    chuck_ccpk=np.append(chuck_ccpk,'All')
    tool1_ccpk=np.append(tool1_ccpk,'All')
    xx=df_lamda.loc[~df_lamda.new_outputs_1.isnull()]
    mean_list=np.array(xx['new_outputs_1'])
#    mean_list=xx[np.logical_not(np.isnan(xx))]
    ccpk=Cpk(mean_list,ucl,lcl)
    model1_ccpk=np.append(model1_ccpk,ccpk)
    lamda_simu=np.append(lamda_simu,lamda)
    
    
ccpk_sum={'tool1':tool1_ccpk,'CCpk':model1_ccpk,'lamda':lamda_simu}
ccpk_df=pd.DataFrame(ccpk_sum)
 
#        
ccpk_df.to_csv('20200422_55Metal.csv')

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

...

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si

C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\edmondchang\AppData\Local\Continuum\anaconda3\lib\si